In [ ]:
import random
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

In [ ]:
import random, calendar
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import datetime

# 1) helper to make N random datetimes in a single month
def random_dates_in_month(year, month, n):
    days_in_month = calendar.monthrange(year, month)[1]
    return [datetime(year, month, random.randint(1, days_in_month))
            for _ in range(n)]

# 2) sample 10 dates in June 2025
arr = random_dates_in_month(2025, 6, 20)


In [ ]:
def bubble_sort(arr):
    a = arr.copy()
    states = [a.copy()]
    swap_count = 0
    n = len(a)
    for i in range(n):
        for j in range(0, n - i - 1):
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
                swap_count += 1
                states.append(a.copy())
    return a, swap_count, states

def quick_sort(arr):
    a = arr.copy()
    states = [a.copy()]
    swap_count = 0

    def _quick_sort(low, high):
        nonlocal swap_count
        if low < high:
            p = partition(low, high)
            _quick_sort(low, p - 1)
            _quick_sort(p + 1, high)

    def partition(low, high):
        nonlocal swap_count
        pivot = a[high]
        i = low
        for j in range(low, high):
            if a[j] < pivot:
                a[i], a[j] = a[j], a[i]
                swap_count += 1
                states.append(a.copy())
                i += 1
        a[i], a[high] = a[high], a[i]
        swap_count += 1
        states.append(a.copy())
        return i

    _quick_sort(0, len(a) - 1)
    return a, swap_count, states

def heap_sort(arr):
    a = arr.copy()
    n = len(a)
    states = [a.copy()]
    swap_count = 0

    def heapify(n, i):
        nonlocal swap_count
        largest = i
        left = 2 * i + 1
        right = 2 * i + 2

        if left < n and a[left] > a[largest]:
            largest = left
        if right < n and a[right] > a[largest]:
            largest = right
        if largest != i:
            a[i], a[largest] = a[largest], a[i]
            swap_count += 1
            states.append(a.copy())
            heapify(n, largest)

    for i in range(n // 2 - 1, -1, -1):
        heapify(n, i)

    for i in range(n - 1, 0, -1):
        a[i], a[0] = a[0], a[i]
        swap_count += 1
        states.append(a.copy())
        heapify(i, 0)

    return a, swap_count, states

def insertion_sort(arr):
    a = arr.copy()
    states = [a.copy()]
    swap_count = 0
    n = len(a)
    for i in range(1, n):
        key = a[i]
        j = i - 1
        while j >= 0 and a[j] > key:
            a[j + 1] = a[j]
            swap_count += 1
            states.append(a.copy())
            j -= 1
        a[j + 1] = key
        swap_count += 1
        states.append(a.copy())
    return a, swap_count, states

def merge_sort(arr):
    a = arr.copy()
    states = [a.copy()]
    swap_count = 0

    def _merge_sort(left: int, right: int):
        nonlocal swap_count
        if left >= right:
            return

        mid = (left + right) // 2
        _merge_sort(left, mid)
        _merge_sort(mid + 1, right)

        temp = []
        i, j = left, mid + 1
        while i <= mid and j <= right:
            if a[i] <= a[j]:
                temp.append(a[i])
                i += 1
            else:
                temp.append(a[j])
                j += 1

        while i <= mid:
            temp.append(a[i])
            i += 1
        while j <= right:
            temp.append(a[j])
            j += 1

        for idx, val in enumerate(temp, start=left):
            a[idx] = val
            swap_count += 1
            states.append(a.copy())

    _merge_sort(0, len(a) - 1)
    return a, swap_count, states

def animate_sort(states, title):
    fig, ax = plt.subplots()
    bars = ax.bar(range(len(states[0])), states[0], align="edge", color='skyblue')
    ax.set_title(title)
    ax.set_xlim(0, len(states[0]))
    ax.set_ylim(0, max(states[0]) * 1.1)
    text = ax.text(0.02, 0.95, "", transform=ax.transAxes)

    def update(frame):
        for bar, val in zip(bars, states[frame]):
            bar.set_height(val)
        text.set_text(f"Step: {frame}/{len(states)-1}")
        return bars, text

    ani = animation.FuncAnimation(fig, update, frames=range(len(states)), interval=200, repeat=False)
    plt.close(fig)  # Prevents duplicate static image in some environments.
    return ani

In [ ]:
def animate_dates(states, title):
    fig, ax = plt.subplots()
    # zero-base
    base = min(d.toordinal() for d in states[0])
    heights0 = [(d.toordinal() - base) for d in states[0]]

    bars = ax.bar(range(len(states[0])), heights0, align="edge", color="skyblue")

    # create a Text for each bar, centered horizontally and vertically
    labels = []
    for i, bar in enumerate(bars):
        x = bar.get_x() + bar.get_width()/2
        y = bar.get_height()/2
        txt = ax.text(x, y,
                      states[0][i].strftime("%Y-%m-%d"),
                      ha="center", va="center", rotation=90, fontsize=8)
        labels.append(txt)

    ax.set_title(title)
    ax.set_xlim(0, len(states[0]))
    ax.set_ylim(0, max(heights0)*1.2)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    def update(frame):
        curr = states[frame]
        heights = [(d.toordinal() - base) for d in curr]
        for bar, h, lbl, date in zip(bars, heights, labels, curr):
            bar.set_height(h)
            # reposition label vertically & update text
            lbl.set_text(date.strftime("%Y-%m-%d"))
            lbl.set_y(h/2)
        return list(bars) + labels

    ani = animation.FuncAnimation(fig, update,
                                  frames=len(states),
                                  interval=300, repeat=False)
    plt.close(fig)
    return ani

# usage:
sorted_dates, swaps, states = bubble_sort(arr)
ani = animate_dates(states, f"Bubble Sort – June 2025 (swaps={swaps})")
from matplotlib.animation import PillowWriter
ani.save(
    'bubble_sort.gif',
    writer=PillowWriter(fps=5),
    dpi=100
)
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
_, swaps, quick_sort_states = quick_sort(arr)
ani_quick_sort = animate_dates(quick_sort_states, f"quick Sort (Swaps: {swaps})")
from matplotlib.animation import PillowWriter
ani_quick_sort.save(
    'quick_sort.gif',
    writer=PillowWriter(fps=5),
    dpi=100
)
from IPython.display import HTML
HTML(ani_quick_sort.to_jshtml())

In [ ]:
_, swaps, heap_sort_states = heap_sort(arr)
ani_heap_sort = animate_dates(heap_sort_states, f"heap_sort (Swaps: {swaps})")
from matplotlib.animation import PillowWriter
ani_heap_sort.save(
    'heap_sort.gif',
    writer=PillowWriter(fps=5),
    dpi=100
)
from IPython.display import HTML
HTML(ani_heap_sort.to_jshtml())

In [ ]:
_, swaps, insertion_sorted_states = insertion_sort(arr)
ani_insertion_sort = animate_dates(insertion_sorted_states, f"insertion_sort (Swaps: {swaps})")
from matplotlib.animation import PillowWriter
ani_insertion_sort.save(
    'insertion_sort.gif',
    writer=PillowWriter(fps=5),
    dpi=100
)
from IPython.display import HTML
HTML(ani_insertion_sort.to_jshtml())

In [ ]:
_, swaps, merge_sorted_states = merge_sort(arr)
ani_merge_sort = animate_dates(merge_sorted_states, f"merge sort (Swaps: {swaps})")
from matplotlib.animation import PillowWriter
ani_merge_sort.save(
    'merge_sort.gif',
    writer=PillowWriter(fps=5),
    dpi=100
)
from IPython.display import HTML
HTML(ani_merge_sort.to_jshtml())

In [ ]:
import time
import random
import tracemalloc
import pandas as pd

def benchmark_sorting(n, trials=100):
    results = {
        "Algorithm": [],
        "Average Swaps": [],
        "Average Time (ms)": [],
        "Average Memory (bytes)": []
    }
    algorithms = [
        ("BubbleSort", bubble_sort),
        ("QuickSort", quick_sort),
        ("HeapSort", heap_sort),
        ("InsertionSort", insertion_sort),
        ("MergeSort", merge_sort),
    ]
    for name, sort_func in algorithms:
        total_swaps = 0
        total_time = 0
        total_memory = 0
        for _ in range(trials):
            arr = random.sample(range(1, n + 1), n)
            tracemalloc.start()
            start = time.time()
            # Assuming sort_func returns (sorted_array, swaps, extra_metric)
            _, swaps, _ = sort_func(arr)
            end = time.time()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            total_swaps += swaps
            total_time += (end - start)
            total_memory += peak
        results["Algorithm"].append(name)
        results["Average Swaps"].append(total_swaps / trials)
        results["Average Time (ms)"].append((total_time / trials) * 1000)
        results["Average Memory (bytes)"].append(total_memory / trials)
    return pd.DataFrame(results)

# Run benchmarks on arrays of 100 elements.
df_benchmark = benchmark_sorting(100, trials=100)
print(df_benchmark)

In [10]:
import matplotlib.pyplot as plt
import networkx as nx
import imageio

def hierarchy_pos(G, root=0, width=1., vert_gap=0.2, vert_loc=0, xcenter=0.5):
    def _hierarchy_pos(G, root, leftmost, rightmost, vert_loc, xcenter, pos):
        pos[root] = (xcenter, vert_loc)
        children = list(G.successors(root))
        if children:
            dx = (rightmost - leftmost) / len(children)
            nextx = leftmost + dx/2
            for child in children:
                pos = _hierarchy_pos(G, child, nextx-dx/2, nextx+dx/2,
                                     vert_loc-vert_gap, nextx, pos)
                nextx += dx
        return pos
    return _hierarchy_pos(G, root, 0, width, vert_loc, xcenter, {})

def plot_heap(arr):
    n = len(arr)
    G = nx.DiGraph()
    for i, val in enumerate(arr):
        G.add_node(i, label=val)
        left, right = 2*i+1, 2*i+2
        if left < n:  G.add_edge(i, left)
        if right < n: G.add_edge(i, right)
    pos = hierarchy_pos(G, 0)
    fig, ax = plt.subplots(figsize=(4, 3))
    labels = {i: arr[i] for i in range(n)}
    nx.draw(G, pos, ax=ax, labels=labels, with_labels=True,
            node_color='lightblue', node_size=1200, font_size=9)
    ax.axis('off')
    return fig

def heapify(arr, n, i, snaps):
    largest = i
    l, r = 2*i+1, 2*i+2
    if l < n and arr[l] > arr[largest]:
        largest = l
    if r < n and arr[r] > arr[largest]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest, snaps)
    snaps.append(arr.copy())

def heap_sort(arr):
    n = len(arr)
    snaps = [arr.copy()]
    for i in range(n//2 - 1, -1, -1):
        heapify(arr, n, i, snaps)
    for i in range(n-1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]
        snaps.append(arr.copy())
        heapify(arr, i, 0, snaps)
    return snaps

# Generate snapshots
dates = ["5", "2", "4", "1", "3"]
snapshots = heap_sort(dates.copy())

# Create GIF
frames = []
for idx, snap in enumerate(snapshots):
    fig = plot_heap(snap)
    filename = f'frame_{idx}.png'
    fig.savefig(filename, dpi=100)
    plt.close(fig)
    frames.append(imageio.imread(filename))

gif_path = 'heap_sort_animation.gif'
imageio.mimsave(gif_path, frames, duration=0.8)

# Provide path for download
gif_path


/var/folders/0_/14m9hb7n2234bn6kbkgxxvbr0000gn/T/ipykernel_27757/3147598605.py:69: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(filename))


'heap_sort_animation.gif'